In [2]:
import numpy as np

In [86]:
def seq_align(seq1,seq2,match = 1,gap_penalty = 1,mismatch_penalty = 1):
    row_len = len(seq1)+1
    col_len = len(seq2)+1
    score_matrix = np.zeros((row_len,col_len),dtype = int)

    maxScore = -1*np.inf
    maxScore_idx = (0,0)
    base_val = 0
    for i in range(1,row_len):
        for j in range(1,col_len):
            if(seq1[i-1] == seq2[j-1]):
                baseScore = match
            else:
                baseScore = -1*mismatch_penalty
            

            diagonal_score = score_matrix[i-1,j-1] + baseScore
            gap_seq2 = score_matrix[i-1,j] - gap_penalty
            gap_seq1 = score_matrix[i,j-1] - gap_penalty

            score_matrix[i,j] = max(0,diagonal_score,gap_seq2,gap_seq1)

            if(score_matrix[i,j]>maxScore):
                maxScore = score_matrix[i,j]
                maxScore_idx = (i,j)

    return score_matrix,maxScore_idx,maxScore

            

def traceBack(score_matrix,maxScore_idx,sequence1,sequence2):
    (i,j) = maxScore_idx
    seq1 = ""
    seq2 = ""
    while(score_matrix[i,j]!=0):
        
        if((score_matrix[i-1,j-1]>=score_matrix[i-1,j]) and (score_matrix[i-1,j-1]>=score_matrix[i,j-1])):
            
            i = i-1
            j = j-1
            seq1+=sequence1[i]
            seq2+=sequence2[j]
            
        elif((score_matrix[i,j-1]>=score_matrix[i-1,j-1]) and (score_matrix[i,j-1]>=score_matrix[i-1,j])):
            
            j = j-1
            seq1+="-"
            seq2+=sequence2[j]
            
        elif((score_matrix[i-1,j]>=score_matrix[i-1,j-1]) and (score_matrix[i-1,j]>=score_matrix[i,j-1])):
            
            i = i-1
            seq1+=sequence1[i]
            seq2+="-"
            
        
        
    return seq1[::-1],seq2[::-1]
        
def SWLocalAlignment(seq1,seq2,match = 1,gap_penalty = 1,mismatch_penalty = 1):
    score_matrix,maxScore_idx,maxScore = seq_align(seq1,seq2,match,gap_penalty,mismatch_penalty)
    rSeq1,rSeq2 = traceBack(score_matrix,maxScore_idx,seq1,seq2)
    return rSeq1,rSeq2,maxScore



In [87]:
score_matrix,maxScore_idx,maxScore = seq_align('tgcatcgagaccctacgtgac','actagacctagcatcgac')
seq1,seq2 = traceBack(score_matrix,maxScore_idx,'tgcatcgagaccctacgtgac','actagacctagcatcgac')
print(seq1)
print(seq2)
print(maxScore)

agacccta-cgt-gac
agacc-tagcatcgac
8


In [88]:
seq1,seq2,maxScore = SWLocalAlignment('tgcatcgagaccctacgtgac','actagacctagcatcgac')
print(seq1)
print(seq2)
print(maxScore)

agacccta-cgt-gac
agacc-tagcatcgac
8


In [89]:
seq1,seq2,maxScore = SWLocalAlignment('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', gap_penalty=2)
print(seq1)
print(seq2)
print(maxScore)

gcatcga
gcatcga
7


## Tests

In [97]:
def test_scoring(seq1,seq2,maxScore,match = 1,mismatch_penalty = 1,gap_penalty = 1):
    score = 0
    for i in range(len(seq1)):
        if(seq1[i] == seq2[i]):
            score+= match
        elif(seq1[i] == "-" or seq2[i] == "-"):
            score-= gap_penalty
        elif(seq1[i]!=seq2[i]):
            score-=mismatch_penalty
    return maxScore == score



#### Wrote a small test to check if the scores match on the basis of the local alignment

In [98]:
seq_1 = 'gcatcga'
seq_2 = 'gcatcga'
test_scoring(seq_1,seq_2,7,gap_penalty=2)

True

In [100]:
from skbio.alignment import StripedSmithWaterman

In [130]:
SW = StripedSmithWaterman('tgcatcgagaccctacgtgac',gap_open_penalty = 1,match_score = 1,mismatch_score = -1)
SW('actagacctagcatcgac')

{
    'optimal_alignment_score': 8,
    'suboptimal_alignment_score': 2,
    'query_begin': 7,
    'query_end': 20,
    'target_begin': 3,
    'target_end_optimal': 17,
    'target_end_suboptimal': 1,
    'cigar': '3M1I4M1D3M1D3M',
    'query_sequence': 'tgcatcgagaccctacgtgac',
    'target_sequence': 'actagacctagcatcgac'
}

#### The basis of my test is that while the individual alignments could vary the scores of the alignment should be the same. Thus for my gold standard I'm using skbio's Striped Smith Waterman algorithm, I generate random values for match score, mismatch score and mismatch penalty and 2 random sequences of size 20, I keep the gap extension penalty to be the same as gap penalty. If the scores vary an exception is raised else it keeps on iterating.

In [159]:
import random
for i in range(100):
    str1 = ''.join(random.choices(['a','t','g','c'], k=50))
    str2 = ''.join(random.choices(['a','t','g','c'], k=50))
    matchScr = random.randint(1,5)
    gapPenalty = random.randint(1,5)
    mismatchPenalty= random.randint(1,5)
    custom_algo_res = SWLocalAlignment(str1,str2,match = matchScr,gap_penalty = gapPenalty,mismatch_penalty = mismatchPenalty)
    SW = StripedSmithWaterman(str1,gap_open_penalty = gapPenalty,gap_extend_penalty = gapPenalty,match_score = matchScr,mismatch_score = -1*mismatchPenalty)
    test_res = SW(str2)
    if(custom_algo_res[2]!=test_res['optimal_alignment_score']):
        print("Test ",i," error")
        print(str1,str2)
        print(test_res)
        print(matchScr,gapPenalty,mismatchPenalty,custom_algo_res)
        raise Exception
    else:
        print("Test ",i," pass")

Test  0  pass
Test  1  pass
Test  2  pass
Test  3  pass
Test  4  pass
Test  5  pass
Test  6  pass
Test  7  pass
Test  8  pass
Test  9  pass
Test  10  pass
Test  11  pass
Test  12  pass
Test  13  pass
Test  14  pass
Test  15  pass
Test  16  pass
Test  17  pass
Test  18  pass
Test  19  pass
Test  20  pass
Test  21  pass
Test  22  pass
Test  23  pass
Test  24  pass
Test  25  pass
Test  26  pass
Test  27  pass
Test  28  pass
Test  29  pass
Test  30  pass
Test  31  pass
Test  32  pass
Test  33  pass
Test  34  pass
Test  35  pass
Test  36  pass
Test  37  pass
Test  38  pass
Test  39  pass
Test  40  pass
Test  41  pass
Test  42  pass
Test  43  pass
Test  44  pass
Test  45  pass
Test  46  pass
Test  47  pass
Test  48  pass
Test  49  pass
Test  50  pass
Test  51  pass
Test  52  pass
Test  53  pass
Test  54  pass
Test  55  pass
Test  56  pass
Test  57  pass
Test  58  pass
Test  59  pass
Test  60  pass
Test  61  pass
Test  62  pass
Test  63  pass
Test  64  pass
Test  65  pass
Test  66  pass
Test 